In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from librep.datasets.har.loaders import (
    MotionSense_BalancedView20HZ
)

2022-11-29 10:39:20.058593: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 10:39:20.296059: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 10:39:20.296100: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-29 10:39:21.616033: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [5]:
from librep.transforms.topo_ae import (
    TopologicalDimensionalityReduction,
    CustomTopoDimRedTransform
)
from librep.transforms import UMAP
from sklearn.manifold import TSNE
from experiments.Topological_ae.Experiment_utils import *
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

## Reading model

In [13]:
import pickle
import matplotlib.pyplot as plt
from tabulate import tabulate
# Variables to reuse
save_ids = range(5)
model_name = 'ConvolutionalAutoencoder_custom_dim3'
model_dim = 10
model_epc = 500
model_lams = [0, 0.1, 0.01, 1, 10, 100]

data = {
    'Modelo': ['Conv_custom_dim3']*len(model_lams),
    'Lambda': model_lams,
    'RF-ACC': [],
    'RF-F1': [],
    'SVC-ACC': [],
    'SVC-F1': [],
    'KNN-ACC': [],
    'KNN-F1': [],
    }
for model_lam in model_lams:
    rf_acc = []
    rf_f1 = []
    svc_acc = []
    svc_f1 = []
    knn_acc = []
    knn_f1 = []
    recon_error = []
    for save_id in save_ids:
        filename = 'data/{}_{}_{}_{}_{}.pkl'.format(model_name, model_lam, model_dim, model_epc, save_id)
        filehandler = open(filename, 'rb')
        model = pickle.load(filehandler)
        filehandler.close()
        recon_error.append(model.train_recon_error[-1])

        filename = 'data/{}_{}_{}_{}_{}.acc_f1.pkl'.format(model_name, model_lam, model_dim, model_epc, save_id)
        filehandler = open(filename, 'rb')
        evaluation = pickle.load(filehandler)
        filehandler.close()
        rf_acc.append(evaluation['RF-ACC'])
        rf_f1.append(evaluation['RF-F1'])
        svc_acc.append(evaluation['SVC-ACC'])
        svc_f1.append(evaluation['SVC-F1'])
        knn_acc.append(evaluation['KNN-ACC'])
        knn_f1.append(evaluation['KNN-F1'])
    print(model_lam, rf_acc)
    data['RF-ACC'].append("{:.4f}±{:.4f}".format(np.mean(rf_acc), np.std(rf_acc)))
    data['RF-F1'].append("{:.4f}±{:.4f}".format(np.mean(rf_f1), np.std(rf_f1)))
    data['SVC-ACC'].append("{:.4f}±{:.4f}".format(np.mean(svc_acc), np.std(svc_acc)))
    data['SVC-F1'].append("{:.4f}±{:.4f}".format(np.mean(svc_f1), np.std(svc_f1)))
    data['KNN-ACC'].append("{:.4f}±{:.4f}".format(np.mean(knn_acc), np.std(knn_acc)))
    data['KNN-F1'].append("{:.4f}±{:.4f}".format(np.mean(knn_f1), np.std(knn_f1)))
print(tabulate(data, headers='keys', tablefmt='fancy_grid'))

0 [0.8963725490196077, 0.8661764705882353, 0.8373529411764705, 0.8418627450980392, 0.9009803921568625]
0.1 [0.9171568627450981, 0.8994117647058824, 0.8698039215686274, 0.9031372549019607, 0.8492156862745098]
0.01 [0.842156862745098, 0.8896078431372549, 0.866764705882353, 0.8660784313725491, 0.8784313725490195]
1 [0.8835294117647059, 0.8558823529411764, 0.8665686274509804, 0.8697058823529412, 0.889313725490196]
10 [0.8957843137254903, 0.8496078431372549, 0.8326470588235294, 0.8470588235294118, 0.894313725490196]
100 [0.8698039215686274, 0.8650980392156862, 0.8683333333333334, 0.8005882352941176, 0.8427450980392157]
╒══════════════════╤══════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╕
│ Modelo           │   Lambda │ RF-ACC        │ RF-F1         │ SVC-ACC       │ SVC-F1        │ KNN-ACC       │ KNN-F1        │
╞══════════════════╪══════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════